# Country Name Cleaning and ISO Code Mapping

This notebook processes a list of raw, uncleaned country names from a survey dataset.  
The goal is to:
1. Match each entry to its official English country name,
2. Identify any unrecognized entries for manual correction,
3. Automatically assign ISO Alpha-2 codes (2-letter country codes).


## Step 1: Load raw list of unique countries

We load the CSV exported from Excel (`unique_countries.csv`) that contains only the raw, uncleaned country names.

In [3]:
# Import required libraries
import pandas as pd
import pycountry

# Load CSV with the list of raw, uncleaned country names
df = pd.read_csv('unique_countries.csv')
df.head()  # preview the first rows


,Raw Country
0,the States
1,UK
2,holland
3,USA
4,uk


## Step 2: Generate initial country mapping

We use the `pycountry` library to:
- Try to match each raw entry to its official English country name,
- Generate an initial ISO Alpha-2 code where possible,
- Mark any unrecognized entries as `"Unknown"` for later manual review.

The result is saved as `country_mapping.csv`, which we will manually review in the next step.

In [4]:
# Function to match each country name to its official name and ISO code
def get_country_info(name):
    try:
        country = pycountry.countries.lookup(name)
        return country.name, country.alpha_2
    except LookupError:
        return "Unknown", "Unknown"

# Apply the function and create new columns
df[['Cleaned Country', 'ISO Code']] = df['Raw Country'].apply(
    lambda x: pd.Series(get_country_info(str(x)))
)

# Save the mapping table
df.to_csv('country_mapping.csv', index=False)
print("Step 2 complete: country_mapping.csv created.")


Step 2 complete: country_mapping.csv created.


## Step 3: Manual correction of country names

Some entries may remain as `"Unknown"` because `pycountry` could not recognize them  
(e.g., due to slang, typos, or uncommon labels like `"maple land"` or `"ooo south africa!"`).

At this stage:
1. Open `country_mapping.csv` in Excel,
2. Correct `"Unknown"` entries by replacing them with official English country names,
3. Save the corrected file (keep the same name).


## Step 3: Manual correction of country names

Some entries may remain as `"Unknown"` because `pycountry` could not recognize them  
(e.g., due to slang, typos, or uncommon labels like `"maple land"` or `"ooo south africa!"`).

At this stage:
1. Open `country_mapping.csv` in Excel,
2. Correct `"Unknown"` entries by replacing them with official English country names,
3. Save the corrected file (keep the same name).

In [6]:
import pandas as pd
import pycountry

# Reload the corrected mapping file (load all columns, not just selected ones)
df = pd.read_csv('country_mapping.csv', sep=';')

# Function to fetch ISO Alpha-2 code for each cleaned country name
def get_iso_code(name):
    try:
        country = pycountry.countries.lookup(name)
        return country.alpha_2
    except LookupError:
        return "Unknown"

# Update ISO Code column (or create it if missing)
df['ISO Code'] = df['Cleaned Country'].apply(get_iso_code)

# Save the final cleaned mapping with all columns preserved
df.to_csv('country_mapping_final.csv', index=False)
print("Step 4 complete: Final mapping saved as country_mapping_final.csv (all columns kept).")

Step 4 complete: Final mapping saved as country_mapping_final.csv (all columns kept).
